## Prepare data

In [1]:
import torch
training_device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
training_device
import numpy as np

In [2]:
#!pip install evaluate datasets transformers accelerate==1.9.0 wandb safetensors==0.4.3

  Using cached gitpython-3.1.45-py3-none-any.whl.metadata (13 kB)
  Using cached gitdb-4.0.12-py3-none-any.whl.metadata (1.2 kB)
  Using cached smmap-5.0.2-py3-none-any.whl.metadata (4.3 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 88.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 148.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 561.5/561.5 kB 60.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 100.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 129.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.4/22.4 MB 118.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 66.6 MB/s eta 0:00:00
Using cached gitpython-3.1.45-py3-none-any.whl (208 kB)
Using cached gitdb-4.0.12-py3-none-any.whl (62 kB)
Using cached smmap-5.0.2-py3-none-any.whl (24 kB)
  Attempting uninstall: fsspec━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━  2/25 [sentry-sdk]
    Found exist

In [3]:
import pandas as pd
from datasets import load_dataset, Dataset, DatasetDict

df = pd.read_csv("data/coh_data.csv")
df.head()

,recordId,gpt4o_judge_score,nova_judge_score,llama3_judge_score,majority_value,agreement_percentage,writing_id,task_id,level_title,cefr_level,ef_level,activity_instructions,student_submission
0,CALL0021715,0.0,0.0,0.0,0.0,100.000000,295f109e-cfeb-48ce-82ba-ce6f2bf423d2,d491686c-8998-4062-bd13-27562deecdbe,Telecommunications,B1,NaN,A client who is thinking about expanding into ...,N n n n n n. N nn n. N n n. N. NN n n n nn n n...
1,CALL0019540,0.0,0.0,0.0,0.0,100.000000,8216749c-2498-4ff2-a241-d66d650689c0,2e74edfb-ed33-4ca0-b50c-d5c0ebda76b7,Logistics,B1,NaN,Now write your counter-proposal. Remember to a...,df f fws wf f wf wf wef we fw fw ef wef ew few...
2,CALL0012294,2.0,NaN,2.0,2.0,66.666667,dd1cf6a9-aed3-47cf-b094-8a86988c79ae,9032f3ed-5da1-4efe-b4dd-c9470bb35d65,16-Upper Advanced,C2,16.0,Read an email from your friend and give advice...,"Hey don give up so quickly, give time some tim..."
3,CALL0005503,4.0,4.0,3.0,4.0,66.666667,a2376227-7402-48ee-ad9a-f909f372ccd5,493dffa9-7513-4408-be4b-5b010a1bf051,10-Upper Intermediate,B2,10.0,"Your manager sent you an email, asking you to ...",Company Update Presentation Outline\n\nGreet e...
4,CALL0007695,0.0,0.0,0.0,0.0,100.000000,85cda5c8-7fe7-46ab-83ba-60c3e09c6afd,d040caa5-7bd8-412c-aee2-3d9010b91821,12-Upper Intermediate,B2,12.0,You're going to reply to Alice's blog post abo...,"Hi Team,\n\nHope you are doing good and it was..."


In [4]:
df["task_id"].value_counts()

task_id
cfb893f1-bee1-4894-99f3-43af284493a5    205
5c44f26d-2f5c-4039-bcf5-378754d2fe4e    194
81ee255e-d4fa-4f81-b77f-dfcf3b5cf541    187
5e5c32c9-cdbe-4b4c-9e83-aa4b8b0ec26a    186
7e62b6a6-8685-4a0b-b0b2-d902bed0c567    186
                                       ... 
511ccd73-9db8-45e5-8f85-10e244841ce2     26
9cf6c0c9-e87b-4e02-84c9-11266ea51d28     24
178cfadd-4162-4b5b-aaee-97a1bdcd4d98     24
e5780a8d-f6ef-4218-9d34-74e064c3e413     19
d2196935-4153-4f5c-a1d2-dae31174ae55     17
Name: count, Length: 161, dtype: int64

In [5]:
df['text'] = (
    "Prompt Level: " + df['level_title'].astype(str) +
    " [SEP] Prompt: " + df['activity_instructions'] +
    " [SEP] Response: " + df['student_submission']
)

In [6]:
df.head()

,recordId,gpt4o_judge_score,nova_judge_score,llama3_judge_score,majority_value,agreement_percentage,writing_id,task_id,level_title,cefr_level,ef_level,activity_instructions,student_submission,text
0,CALL0021715,0.0,0.0,0.0,0.0,100.000000,295f109e-cfeb-48ce-82ba-ce6f2bf423d2,d491686c-8998-4062-bd13-27562deecdbe,Telecommunications,B1,NaN,A client who is thinking about expanding into ...,N n n n n n. N nn n. N n n. N. NN n n n nn n n...,Prompt Level: Telecommunications [SEP] Prompt:...
1,CALL0019540,0.0,0.0,0.0,0.0,100.000000,8216749c-2498-4ff2-a241-d66d650689c0,2e74edfb-ed33-4ca0-b50c-d5c0ebda76b7,Logistics,B1,NaN,Now write your counter-proposal. Remember to a...,df f fws wf f wf wf wef we fw fw ef wef ew few...,Prompt Level: Logistics [SEP] Prompt: Now writ...
2,CALL0012294,2.0,NaN,2.0,2.0,66.666667,dd1cf6a9-aed3-47cf-b094-8a86988c79ae,9032f3ed-5da1-4efe-b4dd-c9470bb35d65,16-Upper Advanced,C2,16.0,Read an email from your friend and give advice...,"Hey don give up so quickly, give time some tim...",Prompt Level: 16-Upper Advanced [SEP] Prompt: ...
3,CALL0005503,4.0,4.0,3.0,4.0,66.666667,a2376227-7402-48ee-ad9a-f909f372ccd5,493dffa9-7513-4408-be4b-5b010a1bf051,10-Upper Intermediate,B2,10.0,"Your manager sent you an email, asking you to ...",Company Update Presentation Outline\n\nGreet e...,Prompt Level: 10-Upper Intermediate [SEP] Prom...
4,CALL0007695,0.0,0.0,0.0,0.0,100.000000,85cda5c8-7fe7-46ab-83ba-60c3e09c6afd,d040caa5-7bd8-412c-aee2-3d9010b91821,12-Upper Intermediate,B2,12.0,You're going to reply to Alice's blog post abo...,"Hi Team,\n\nHope you are doing good and it was...",Prompt Level: 12-Upper Intermediate [SEP] Prom...


In [7]:
df = df[["text", "task_id", "level_title", "majority_value"]]
df = df.rename(columns={'majority_value': 'label'})
df.head()

,text,task_id,level_title,label
0,Prompt Level: Telecommunications [SEP] Prompt:...,d491686c-8998-4062-bd13-27562deecdbe,Telecommunications,0.0
1,Prompt Level: Logistics [SEP] Prompt: Now writ...,2e74edfb-ed33-4ca0-b50c-d5c0ebda76b7,Logistics,0.0
2,Prompt Level: 16-Upper Advanced [SEP] Prompt: ...,9032f3ed-5da1-4efe-b4dd-c9470bb35d65,16-Upper Advanced,2.0
3,Prompt Level: 10-Upper Intermediate [SEP] Prom...,493dffa9-7513-4408-be4b-5b010a1bf051,10-Upper Intermediate,4.0
4,Prompt Level: 12-Upper Intermediate [SEP] Prom...,d040caa5-7bd8-412c-aee2-3d9010b91821,12-Upper Intermediate,0.0


In [8]:
# Apply the mapping to the 'labels' column
#df['label'] = df['label'].map(label_mapping)
df.dropna(subset=['label', 'text'], inplace=True)
df.reset_index(drop=True, inplace=True)

df.head()

,text,task_id,level_title,label
0,Prompt Level: Telecommunications [SEP] Prompt:...,d491686c-8998-4062-bd13-27562deecdbe,Telecommunications,0.0
1,Prompt Level: Logistics [SEP] Prompt: Now writ...,2e74edfb-ed33-4ca0-b50c-d5c0ebda76b7,Logistics,0.0
2,Prompt Level: 16-Upper Advanced [SEP] Prompt: ...,9032f3ed-5da1-4efe-b4dd-c9470bb35d65,16-Upper Advanced,2.0
3,Prompt Level: 10-Upper Intermediate [SEP] Prom...,493dffa9-7513-4408-be4b-5b010a1bf051,10-Upper Intermediate,4.0
4,Prompt Level: 12-Upper Intermediate [SEP] Prom...,d040caa5-7bd8-412c-aee2-3d9010b91821,12-Upper Intermediate,0.0


In [9]:
df["label"].value_counts()

label
5.0    5302
4.0    4056
0.0    3881
2.0    2699
3.0    1925
1.0    1048
Name: count, dtype: int64

In [10]:
ds = Dataset.from_pandas(df)
ds

Dataset({
    features: ['text', 'task_id', 'level_title', 'label'],
    num_rows: 18911
})

In [11]:
from datasets import ClassLabel, Value, Sequence
new_features = ds.features.copy()
new_features["label"] = ClassLabel(names=[0, 1, 2, 3, 4, 5])
ds = ds.cast(new_features)

# Step 1: Initial train/test split with stratification
train_test_ds = ds.train_test_split(test_size=0.20, seed=20)

# Step 2: Split the test set into half test, half validation
test_valid_split = train_test_ds['test'].train_test_split(test_size=0.5, seed=20)

# Step 3: Combine everything into a single DatasetDict
ds = DatasetDict({
    'train': train_test_ds['train'],
    'test': test_valid_split['train'],    # This becomes the test set
    'validation': test_valid_split['test']  # This becomes the validation set
})
ds

Casting the dataset:   0%|          | 0/18911 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'task_id', 'level_title', 'label'],
        num_rows: 15128
    })
    test: Dataset({
        features: ['text', 'task_id', 'level_title', 'label'],
        num_rows: 1891
    })
    validation: Dataset({
        features: ['text', 'task_id', 'level_title', 'label'],
        num_rows: 1892
    })
})

In [12]:
# Verify label distribution
from collections import Counter

print("Train label counts:", Counter(ds['train']['label']))
print("Test label counts:", Counter(ds['test']['label']))
print("Validation label counts:", Counter(ds['validation']['label']))

Train label counts: Counter({5: 4242, 4: 3200, 0: 3119, 2: 2161, 3: 1561, 1: 845})
Test label counts: Counter({5: 539, 4: 416, 0: 373, 2: 275, 3: 174, 1: 114})
Validation label counts: Counter({5: 521, 4: 440, 0: 389, 2: 263, 3: 190, 1: 89})


In [13]:
import json

# Fonction utilitaire pour sauvegarder un split en JSONL
def save_split_to_jsonl(dataset_split, filename):
    with open(filename, 'w', encoding='utf-8') as f:
        for record in dataset_split:
            f.write(json.dumps(record, ensure_ascii=False) + '\n')

# Sauvegarde des trois splits
save_split_to_jsonl(ds['train'], 'data/train.jsonl')
save_split_to_jsonl(ds['test'], 'data/test.jsonl')
save_split_to_jsonl(ds['validation'], 'data/validation.jsonl')

## Roberta SFT

In [14]:
import numpy as np 
import evaluate
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, cohen_kappa_score, classification_report
from scipy.stats import pearsonr

metric = evaluate.load("accuracy")

In [15]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)  # Convertir les logits en classes prédictes

    # 🎯 Exactitude (Accuracy)
    accuracy = accuracy_score(labels, predictions)

    # 🎯 Précision, Rappel et F1-score (pondérés)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average="weighted")

    # 🎯 Score de Cohen's Kappa (pondéré)
    cohen_kappa = cohen_kappa_score(labels, predictions, weights="quadratic")

    # 🎯 Corrélation de Pearson
    pearson_corr, _ = pearsonr(labels, predictions)  # Retourne (coef, p-valeur), on garde seulement coef

     # 🎯 Classification Report
    class_report = classification_report(labels, predictions, output_dict=True)  # Get a dictionary of the report


    return {
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1": f1,
        "cohen_kappa": cohen_kappa,
        "pearson_corr": pearson_corr,
        "classification_report": class_report  # Add classification report to the return
    }


In [16]:
from datasets import load_dataset
from datasets import DatasetDict, Dataset

# Charger les fichiers JSONL en DatasetDict
dataset = DatasetDict({
    "train": load_dataset("json", data_files="data/train.jsonl")["train"],
    "test": load_dataset("json", data_files="data/test.jsonl")["train"],
    "valid": load_dataset("json", data_files="data/validation.jsonl")["train"]
})

print(dataset)

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'task_id', 'level_title', 'label'],
        num_rows: 15128
    })
    test: Dataset({
        features: ['text', 'task_id', 'level_title', 'label'],
        num_rows: 1891
    })
    valid: Dataset({
        features: ['text', 'task_id', 'level_title', 'label'],
        num_rows: 1892
    })
})


In [17]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("FacebookAI/roberta-large")

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

In [18]:
dataset["train"][0]

{'text': "Prompt Level: 15-Advanced [SEP] Prompt: Read Ken's email and then email Josh and Ken a third option. Use your imagination. You can create any facts about your city that will support your proposal. Type in the input box. Write 150-200 words. [SEP] Response: Hello Josh, Ken,\nI hope I can propose third option for you. It should be the best compromise of two options which you have considered so long.\nThe idea is based on the fact that two third of our current shop is not seen by our customers. We have cloth making area and administration area in our shop. They can be moved to south end of the town while keeping showroom and measurement space in downtown.\nThis kind of separation cannot be done in the past because our cloth making section needs to have close contact with measurement service section. However, thanks to IT development, now we can share measurement data immediately and we can start making cloth just after measurement, even though cloth making room is far from our s

In [19]:
tok_test = tokenizer(dataset["train"][1]["text"], max_length=256, truncation=True)
tok_test

{'input_ids': [0, 35396, 3320, 12183, 35, 508, 12, 44694, 646, 3388, 510, 742, 42944, 35, 9387, 9, 402, 7735, 6, 3571, 50, 6269, 14, 1102, 7, 47, 682, 6, 8, 6190, 24, 11, 41, 1047, 7, 10, 1441, 4, 5293, 24, 2679, 8, 42690, 6, 8, 2145, 7, 1649, 13, 44811, 5135, 4, 7773, 11, 5, 8135, 2233, 4, 21062, 3982, 12, 2619, 1617, 4, 646, 3388, 510, 742, 19121, 35, 12191, 1560, 6, 50118, 50118, 1185, 351, 75, 679, 99, 1102, 7, 162, 94, 363, 328, 38, 21, 11963, 15, 5, 26711, 2494, 10, 1569, 137, 3630, 77, 38, 1317, 10, 7735, 6496, 31, 20806, 4, 38, 34576, 89, 21, 5907, 1207, 11, 5, 2853, 1553, 27699, 6, 98, 38, 1276, 7, 8861, 24, 4, 125, 172, 6, 10, 92, 543, 2058, 2369, 37747, 162, 4, 38, 1224, 160, 5, 30016, 6, 342, 15, 127, 3581, 268, 8, 3203, 62, 7, 5, 371, 1929, 4, 38, 6536, 15, 5, 1883, 142, 24, 21, 628, 615, 7, 32366, 5670, 1493, 4, 440, 1948, 4, 38, 1381, 7, 1224, 15, 5, 1109, 9, 5, 14106, 6, 53, 24, 399, 75, 173, 4, 22, 713, 1068, 16, 1158, 7, 32580, 225, 162, 1297, 38, 802, 4, 38, 6536, 45

In [20]:
def tokenize_function(examples):
    return tokenizer(
        examples["text"],
        padding="max_length",
        truncation=True,
        max_length=256
    )

In [21]:
tokenized_train = dataset["train"].map(tokenize_function, batched=True)
tokenized_test = dataset["test"].map(tokenize_function, batched=True)
tokenized_valid = dataset["valid"].map(tokenize_function, batched=True)

Map:   0%|          | 0/15128 [00:00<?, ? examples/s]

Map:   0%|          | 0/1891 [00:00<?, ? examples/s]

Map:   0%|          | 0/1892 [00:00<?, ? examples/s]

In [22]:
unique_labels = set(dataset['train']['label'])
num_labels = len(unique_labels)
num_labels

6

In [23]:
from transformers import RobertaForSequenceClassification, TrainingArguments, Trainer
model = RobertaForSequenceClassification.from_pretrained("FacebookAI/roberta-large", num_labels=num_labels)

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at FacebookAI/roberta-large and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
args = TrainingArguments(
    output_dir="../../../model_saved/roberta-large-ft-acc-writing-task-augmented",
    eval_strategy="steps",  # Évaluation aux mêmes intervalles que la sauvegarde
    save_strategy="steps",  # Sauvegarde tous les 500 steps
    save_steps=200,
    eval_steps=200,  # ⚠ IMPORTANT : Évaluation aux mêmes steps
    save_total_limit=4,  # Ne garde que 4 checkpoints max
    learning_rate=2e-5,
    warmup_ratio=0.1,
    lr_scheduler_type="linear", 
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=6,
    weight_decay=0.01,
    load_best_model_at_end=True,  
    metric_for_best_model="f1",
    logging_steps=100,
    fp16=True,
)

In [ ]:
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()

In [ ]:
trainer.evaluate()

## Detailed Evaluation

In [ ]:
list_topic = dataset["valid"]["task_id"]
list_t_set = set(list_topic)
unique_t = (list(list_t_set))

list_level = dataset["valid"]["level_title"]
list_l_set = set(list_level)
unique_l = (list(list_l_set))


In [ ]:
list_r = []

# Assuming 'unique_t' is a list of unique item_ids and 'trainer' is already defined
for t in unique_t:  # Iterate over the first item in unique_t
    sub_ds = tokenized_valid.filter(lambda example: example['task_id'] == t)
    # Get predictions using the trainer
    predictions = trainer.predict(sub_ds)
    # Raw output logits (size [batch_size, num_classes])
    outputs = predictions.predictions
    # Convert logits to predicted class labels (taking the argmax across the classes)
    predicted_labels = np.argmax(outputs, axis=-1)
    ref_label = predictions.label_ids
    # Print or save the predicted classes (this will be a numpy array with the predicted class indices)
    ck = round(cohen_kappa_score(predicted_labels, ref_label, weights="quadratic"), 2)  
    pearson_corr, _ = pearsonr(ref_label, predicted_labels)
    accuracy = accuracy_score(ref_label, predicted_labels)
    precision, recall, f1, _ = precision_recall_fscore_support(ref_label, predicted_labels, average="weighted")

    r = {
        "task_id": t,
        "level_title": sub_ds["level_title"][0],
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1": f1,
        "ck": ck,
        "pearson": pearson_corr,
        "n_samples": len(sub_ds)
    }
    list_r.append(r)

In [ ]:
list_r_level = []

# Assuming 'unique_t' is a list of unique item_ids and 'trainer' is already defined
for l in unique_l:  # Iterate over the first item in unique_t
    sub_ds = tokenized_valid.filter(lambda example: example['level_title'] == l)
    # Get predictions using the trainer
    predictions = trainer.predict(sub_ds)
    # Raw output logits (size [batch_size, num_classes])
    outputs = predictions.predictions
    # Convert logits to predicted class labels (taking the argmax across the classes)
    predicted_labels = np.argmax(outputs, axis=-1)
    ref_label = predictions.label_ids
    # Print or save the predicted classes (this will be a numpy array with the predicted class indices)
    ck = round(cohen_kappa_score(predicted_labels, ref_label, weights="quadratic"), 2)  
    pearson_corr, _ = pearsonr(ref_label, predicted_labels)
    accuracy = accuracy_score(ref_label, predicted_labels)
    precision, recall, f1, _ = precision_recall_fscore_support(ref_label, predicted_labels, average="weighted")

    r = {
        "level_title": sub_ds["level_title"][0],
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1": f1,
        "ck": ck,
        "pearson": pearson_corr,
        "n_samples": len(sub_ds)
    }
    list_r_level.append(r)


In [ ]:
import pandas as pd
df_eval_results = pd.DataFrame(list_r, columns=["task_id", "level_title", "accuracy", "precision", "recall", "f1", "ck", "pearson", "n_samples"])
df_eval_results.head(n=10)

In [ ]:
df_eval_results.to_csv("result_eval_data_roberta_large_writing_task_acc.csv", index=False)

In [ ]:
import pandas as pd
df_eval_results_level = pd.DataFrame(list_r_level, columns=["level_title", "accuracy", "precision", "recall", "f1", "ck", "pearson", "n_samples"])
df_eval_results_level.head(n=20)

In [ ]:
df_eval_results_level.to_csv("result_eval_data_roberta_large_acc_by_level.csv", index=False)

## Onnx Export

In [14]:
#!pip install onnxruntime

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.4/6.4 MB 119.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4/4 [onnxruntime] [onnxruntime]


In [ ]:
from transformers import RobertaTokenizer, RobertaForSequenceClassification, AutoTokenizer
import torch
import os
from onnxruntime.quantization import quantize_dynamic, QuantType

# === CONFIGURATION ===
# Chemin vers ton dossier contenant le .bin et le config.json
model_dir = "model_saved/roberta-large-ft-acc-writing-task-augmented/checkpoint-1800"
onnx_model_path = "model_saved/roberta-large-ft-acc-writing-task-1800.onnx"
quantized_model_path = "model_saved/roberta-large-ft-acc-writing-task-1800-quantized.onnx"

# === ÉTAPE 1 : Charger le modèle et tokenizer ===
model = RobertaForSequenceClassification.from_pretrained(model_dir)
tokenizer = AutoTokenizer.from_pretrained("FacebookAI/roberta-large")
model.eval()

# === ÉTAPE 2 : Préparer un input fictif ===
dummy_text = "Texte d'exemple pour conversion ONNX"
inputs = tokenizer(dummy_text, return_tensors="pt", padding="max_length", max_length=32)

# === ÉTAPE 3 : Exporter vers ONNX ===
torch.onnx.export(
    model,
    (inputs["input_ids"], inputs["attention_mask"]),
    onnx_model_path,
    input_names=["input_ids", "attention_mask"],
    output_names=["logits"],
    dynamic_axes={
        "input_ids": {0: "batch_size", 1: "sequence_length"},
        "attention_mask": {0: "batch_size", 1: "sequence_length"},
        "logits": {0: "batch_size"},
    },
    opset_version=14  # ⬅️ change ici
)

print(f"✅ Modèle exporté en ONNX : {onnx_model_path}")

# === ÉTAPE 4 : Quantization dynamique ===
quantize_dynamic(
    model_input=onnx_model_path,
    model_output=quantized_model_path,
    weight_type=QuantType.QInt8
)

print(f"✅ Modèle quantifié en ONNX : {quantized_model_path}")

In [ ]:
import onnxruntime

onnx_session = onnxruntime.InferenceSession(onnx_model_path)
onnx_session_quant = onnxruntime.InferenceSession(quantized_model_path)

In [17]:
max_length = 256  # Ajuste selon la taille maximale de ton modèle

# Fonction d'inférence ONNX
def onnx_infer(input_texts, onnx_model):
    inputs = tokenizer(input_texts, padding=True, truncation=True, max_length=max_length, return_tensors="pt")
    input_ids = inputs["input_ids"].numpy()
    attention_mask = inputs["attention_mask"].numpy()
    onnx_inputs = {"input_ids": input_ids, "attention_mask": attention_mask}
    onnx_outputs = onnx_model.run(None, onnx_inputs)
    return onnx_outputs[0]

from tqdm import tqdm

def evaluate_with_metrics(dataset, onnx_model, batch_size=16):
    all_logits = []
    all_labels = []

    # tqdm pour afficher la progression sur les batches
    for i in tqdm(range(0, len(dataset), batch_size), desc="Evaluation"):
        batch = dataset[i:i + batch_size]
        texts = batch["text"]
        labels = batch["label"]

        logits = onnx_infer(texts, onnx_model)
        all_logits.extend(logits)
        all_labels.extend(labels)

    all_logits = np.array(all_logits)
    all_labels = np.array(all_labels)

    # 🔥 Appliquer compute_metrics
    metrics = compute_metrics((all_logits, all_labels))
    return metrics

In [ ]:
valid_data = ds["validation"]  # Ou "valid" selon ton dataset
# === Lancer l'évaluation ===
results = evaluate_with_metrics(valid_data, onnx_model=onnx_session)
print("🎯 Evaluation Results ONNX :")
for k, v in results.items():
    if k == "classification_report":
        print("\n📋 Classification Report :")
        for label, metrics in v.items():
            print(f"{label}: {metrics}")
    else:
        print(f"{k}: {v}")

In [ ]:
valid_data = ds["validation"]  # Ou "valid" selon ton dataset
# === Lancer l'évaluation ===
results = evaluate_with_metrics(valid_data, onnx_model=onnx_session_quant)
print("🎯 Evaluation Results ONNX :")
for k, v in results.items():
    if k == "classification_report":
        print("\n📋 Classification Report :")
        for label, metrics in v.items():
            print(f"{label}: {metrics}")
    else:
        print(f"{k}: {v}")

In [27]:
import boto3

# Initialize the S3 client
s3 = boto3.client(
    "s3"
)

# Define your bucket name and desired path in S3

bucket_name = "sagemaker-studio-oxs6vznjds"

s3_key = "writing_task_models/accuracy/model_1800_roberta_large.onnx"  # Change path as needed
# Upload the ONNX file
bucket_path = "sagemaker-studio-oxs6vznjds"
quantized_model_path = "model_saved/roberta-large-ft-acc-writing-task-1800.onnx"

s3.upload_file(quantized_model_path, bucket_path, s3_key)

print(f"✅ ONNX model uploaded to s3://{bucket_name}/{s3_key}")

✅ ONNX model uploaded to s3://sagemaker-studio-oxs6vznjds/writing_task_models/accuracy/model_1800_roberta_large.onnx


## Load Onnx from s3

In [14]:
import boto3
import onnxruntime as ort

# Define S3 bucket and model key
bucket_name = 'sagemaker-studio-oxs6vznjds'
model_key = 'writing_task_models/accuracy/model_1800_roberta_large.onnx'
local_model_path = '/tmp/roberta-large-ft-acc-writing-task-1800.onnx'  # or wherever you want to save temporarily

# Initialize boto3 S3 client
s3 = boto3.client('s3')

# Download the ONNX model from S3 to local path
s3.download_file(bucket_name, model_key, local_model_path)

# Load the ONNX model using onnxruntime
session = ort.InferenceSession(local_model_path)

print("ONNX model loaded successfully.")

ONNX model loaded successfully.


In [16]:
valid_data = ds["validation"]  # Ou "valid" selon ton dataset

In [23]:
results = evaluate_with_metrics(valid_data, onnx_model=session)
print("🎯 Evaluation Results ONNX :")
for k, v in results.items():
    if k == "classification_report":
        print("\n📋 Classification Report :")
        for label, metrics in v.items():
            print(f"{label}: {metrics}")
    else:
        print(f"{k}: {v}")


Evaluation: 100%|██████████| 126/126 [20:57<00:00,  9.98s/it]

🎯 Evaluation Results ONNX :
accuracy: 0.7571072319201995
precision: 0.754690191285516
recall: 0.7571072319201995
f1: 0.7550283590999475
cohen_kappa: 0.9127375223497928
pearson_corr: 0.9133259543888621

📋 Classification Report :
0: {'precision': 0.9378378378378378, 'recall': 0.9353099730458221, 'f1-score': 0.9365721997300944, 'support': 371.0}
1: {'precision': 0.5964912280701754, 'recall': 0.41975308641975306, 'f1-score': 0.4927536231884058, 'support': 81.0}
2: {'precision': 0.7876344086021505, 'recall': 0.7610389610389611, 'f1-score': 0.774108322324967, 'support': 385.0}
3: {'precision': 0.5753424657534246, 'recall': 0.5853658536585366, 'f1-score': 0.5803108808290155, 'support': 287.0}
4: {'precision': 0.5757575757575758, 'recall': 0.5757575757575758, 'f1-score': 0.5757575757575758, 'support': 330.0}
5: {'precision': 0.8321917808219178, 'recall': 0.8820326678765881, 'f1-score': 0.8563876651982378, 'support': 551.0}
accuracy: 0.7571072319201995
macro avg: {'precision': 0.717542549473847